<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900"> **7:** 编排与智能体</font>

欢迎回到课程！在之前的 notebook 中，我们探索了 LLM 服务，部署了能够处理复杂指令并进行对话的模型。我们学习了如何通过各种接口与这些模型进行交互，并简要介绍了 LLM 编排。

现在，我们将更深入地探讨如何编排 LLM 来构建复杂的应用。可以把 LLM 编排看作是指挥一场交响乐，不同的组件——提示词、检索机制、路由策略和工具——都和谐地协作，构建出强大的终端应用。

#### **学习目标：**

到本 notebook 结束时，您将能够：
- 理解如何处理严肃的 LLM 任务，比如长篇推理和生成。
- 学习提升上下文和改善 LLM 输出的检索技术。
- 探索将任务导向适当模型或工具的路由策略。
- 学习如何集成外部工具来扩展 LLM 的能力。
- 开发能够进行推理和迭代行动的智能系统，最终形成 ReAct 循环。


In [ ]:
import requests
from langchain_nvidia_ai_endpoints import ChatNVIDIA

## USE THIS ONE TO START OUT WITH. NOTE IT'S INTENTED USE AS A VISUAL LANGUAGE MODEL FIRST
# model_path="http://localhost:9000/v1"
## USE THIS ONE FOR GENERAL USE AS A SMALL-BUT-PURPOSE CHAT MODEL BEING RAN LOCALLY VIA NIM
model_path="http://nim:8000/v1"
# ## USE THIS ONE FOR ACCESS TO CATALOG OF RUNNING NIM MODELS IN `build.nvidia.com`
# model_path="http://llm_client:9000/v1"

model_name = requests.get(f"{model_path}/models").json().get("data", [{}])[0].get("id")
%env NVIDIA_BASE_URL=$model_path
%env NVIDIA_DEFAULT_MODE=open

if "llm_client" in model_path:
    model_name = "meta/llama-3.1-70b-instruct"

llm = ChatNVIDIA(model=model_name, base_url=model_path, max_tokens=5000, temperature=0)

<hr>
<br>

## **7.1：** 构建 LLM 工作流

提示工程是设计输入的艺术与科学，旨在引导 LLM 生成期望的输出。由于这些模型本质上是**随机鹦鹉（stochastic parrots）**——也就是说，它们基于输入和训练数据输出概率性响应——您可以预期某些任务对最强大的模型来说会容易得多。幸运的是，我们确切知道这些模型最擅长的是什么：**“摘要”或合成类型的任务**。

**擅长短文本生成的原因其实很简单：**
- 长文本生成难以持续追踪，LLM 可能会因自回归抽样的累积误差而偏离方向。
- 大多数开发者希望节省生成长度，更愿意接受过短的响应，而不是意外过长的响应。
- 在聊天应用中（通常是最流行的默认模型，也是最常用的指令格式），较短、简洁和“像摘要一样”的响应在实际场景中更受欢迎。

**话虽如此，长上下文推理却被高度重视：**
- 指定长输出以帮助强化生成风格/格式的能力，从最终用户的角度来看是非常有吸引力的。
- 即使响应的生成先验倾向于短输出，短输出的累积也可能迅速变得非常长。

**基于这些原因，大多数模型往往被训练用于短文本生成并吸收长段的上下文。** 这使得 LLM 非常适合诸如摘要和长文本问答等任务，本质上归结为**知识合成**或**蒸馏（distillation）**问题。这个概念在之前的 notebook 中已经用代码进行了探讨，但在今后的学习中，正式化并牢记这一点是非常重要的。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from chatbot.jupyter_tools import FileLister

import os 
filenames = [v for v in sorted(os.listdir("temp_dir")) if v.endswith(".ipynb")]

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a helpful DLI Chatbot who can request and reason about notebooks."
        " Be as concise as necessary, but follow directions as best as you can."
        " Please help the user out by answering any of their questions and following their instructions."
    )),
    ("human", "Here is the notebook I want you to work with: {full_context}. Remembering this, start the conversation over."),
    ("ai", "Awesome! I will work with this as context and will restart the conversation."),
    ("placeholder", "{messages}")
])

def compute_context(state: dict):
    return FileLister().to_string(files=state.get("filenames"), workdir=".")

pipeline = (
    RunnablePassthrough.assign(full_context = compute_context)
    | chat_prompt 
    | llm 
    | StrOutputParser()
)

chat_state = {
    # "filenames": ["07_intro_agentics.ipynb"],
    # "messages": [("human", "Can you give me a summary of the notebook?")],
    ## Reason about the entire course at once. This will be much slower and does not scale to larger document pools. 
    "filenames": filenames, 
    "messages": [("human", 
        "Can you give me a summary of the course, making sure to mention every notebook?"
        " Do a paragraph per notebook, and finish by explaining big-picture ideas to help an"
        " instructor explain the material and understand which parts of the course to refer to when addressing questions."
    )],
}

short_summary = ""
for chunk in pipeline.stream(chat_state):
    print(chunk, end="")
    short_summary += chunk

<br>

### 摆脱限制

尽管现代模型有其自然倾向，但人们仍然能够通过多种可扩展的方法完成复杂的任务，比如长文本生成甚至更长的上下文摄取。**这些技术依赖于一个关键假设，即 LLM 可以被程序化地、重复地、独立地和并行地调用（服务器部署正好可以实现这些）。**

<div><img src="imgs/data-pipelines.png" width="800"/></div>

#### **迭代生成** 

对话实际上是一个长文本的逐步生成任务（其中一半的生成由人类完成），因此可以一次生成一段任意长度的文档。如果一个 LLM 只能一次输出 5000 个 token，但可以同时推理 100,000 个 token，那是不是可以一步一步生成一整篇文档呢？
- **全上下文：** 如果迭代生成保持在模型的输入上下文限制内，那么保持对话历史、任务历史、文档历史等的全上下文方法可能就足够了。
- **运行状态：** 当长上下文积累不受欢迎时，从之前生成中获取的运行摘要或其它类型的累积历史可能就足够了。这通常被称为 **“迭代优化（iterative refinement）”**，并且有保持**全局上下文**（或至少是回顾上下文，全局上下文可以通过预处理获得）的好处。
- **滑动窗口：** 如果你想要摘要一个文档，只关心**局部上下文**（比如为了边界一致性，boundary consistency），您可以选择放弃整体历史，只考虑一段内容的窗口。这在**翻译任务**中尤其相关，因为你可以在翻译一个文档片段的同时记住之前翻译的部分和后面尚未翻译的部分。

In [ ]:
message_prompt = (
    "Give me a structured summary of the course, making sure to note all sections and points?"
    " This is a one-notebook-at-a-time process, and the next notebook is the one provided in context."
    "\n\nThe following is a running summary of previous notebooks: \n\n{running_summary}\n\n"
    "Output only the summary of the currently-provided notebook, but explain the logical connections to the rest of the course."
    "Make sure the descriptions are also dense in key words that would be useful for searching through the notebooks via a bibliography."
    " Only output the following format, with no other structures, extra newlines, or info not grounded in context:\n<notebook>.ipynb"
    "\n - <Section 1 Name (As Seen In Notebook)>: Decent Description, including frameworks used, important topics, etc."
    "\n - ..."
    "\n - Main Ideas and Relevance To Course: Decent Description, dense with key features/frameworks/topics for bibliographic/semantic lookup."
    "\n - Important Code: Types of syntaxes, variables, etc that are more specific to this notebook, like classes, variables, topics, terms, etc."
    "\n - Connections to previous notebooks: Decent Description, dense with key features/frameworks/topics for bibliographic/semantic lookup."
    "\n - Relevant Images: Brief descriptions of images (i.e. <img src='imgs/url.png'>, no non-image files) with local-scope URLs."
)

running_summaries = []
running_summary = "No summary yes. This is the first notebook."

for name in filenames:
    if not name.endswith(".ipynb"):
        continue
    buffer = ""
    chat_state = {
        "filenames": [name],
        # "messages": [("human", message_prompt.format(running_summary=running_summary))]                    ## Full history
        "messages": [("human", message_prompt.format(running_summary=(running_summaries or ["None"])[-1]))]  ## Sliding window
        # "messages": [("human", message_prompt.format(running_summary="Not provided"))]                     ## No history, 
    }
    for chunk in pipeline.stream(chat_state):
        buffer += chunk
        print(chunk, end="")
    print("\n" + "*" * 84)
    running_summaries += [buffer]
    running_summary = "\n\n".join(running_summaries)

<details>
<summary><b>迭代优化的注意事项：</b></summary>
<ul>
    <li>上面的例子展示了几种不同的策略（全上下文、滑动窗口和无历史）。你会发现前两者生成的提炼（refinement）有所不同，但在当前这个上下文中还区别不大（不过全上下文在处理长文本提炼时会开始出现问题）。无历史也会有一些问题（有部分原因是需要更多信息，还有部分原因是提示词的指令暗示每个 notebook 都是第一个 notebook）。</li>
    <li>注意对话和思维链（逐步思考）推理在本质上也是提炼问题，其响应是从全上下文（例如聊天历史）中提炼出来的，然后作为下一步提炼的新补充。</li>
    <li>记住，LLM 在有“本可以实际生成”的输入时会受益匪浅。幻觉产生的一个常见原因是给 LLM 提供了它在没有充分告知的情况下不应能生成的上下文。</li>
</ul>
</details>

<br>

#### **并行生成**

在某些情况下，迭代生成可能非常慢，或者可能累积到难以处理的上下文或错误。在这些情况下，您可能想要独立地推理输入的较小部分，稍后再将结果组合在一起。
- **规范化：**如果您有一个主要由独立组件组成的任务，可以独立推理它们，并将其进展为标准化的表示，这样在后续处理时会更容易。例如，缩短文档的某些部分、提取重要细节，或者将简短的摘要扩展为更完整的形式。
- **结构化：**在从长文档中提取全局上下文的任务中（即总结），您可以先对一个窗口进行摘要，然后再对这些摘要进行总结，依此类推，直到得到一个完整的摘要。类似的结构化思路包括[**将知识图谱插入值-边-值**](https://neo4j.com/developer-blog/knowledge-graph-llama-nvidia-langchain/)、[**将嵌入插入向量存储以进行语义检索**](https://arxiv.org/abs/2312.10997)、[**以及插入 SQL 数据库**](https://developer.nvidia.com/blog/new-llm-snowflake-arctic-model-for-sql-and-code-generation/)等，都可以在基于大型数据集应用 LLM 的大型应用中实现。
- **集成：**给定 LLM 做某事的目标，可以使用不同方法的集成来独立尝试解决自然语言问题。这些方法的结果可以结合在一起形成最终的推理结果。

In [ ]:
from langchain_core.runnables import RunnableLambda
from functools import partial

message_prompt = (
    " Give me a rigorous summary of only bullet #{section_i} of the outline. Do not summarize any other sections."
    " Output should be a few compact-but-dense paragraphs long and only summarize a fraction of the notebook (bullet {section_i})"
    " such that a reasonable person would be able to understand everything from that section"
    " (while knowing roughly how it ties in with the whole notebook) from the summary." 
)

notebook_chunks = []

def summarize_section(state):
    name, i = state.get("name"), state.get("i")
    print(f"(+{name[1]}.{i})", end="")
    output = pipeline.invoke({
        "filenames": [name],
        "messages": [
            ("human", "Please give me a structured outline of the notebook"), 
            ("ai", state.get("outline")),
            ("human", message_prompt.format(section_i=i))],
    })
    print(f"(-{name[1]}.{i})", end="", flush=True)
    return output

task_keys = []
task_args = []
task_vals = []
num_tasks = 6

for name, outline in zip(filenames, running_summaries):
    task_keys += (nb_keys := [f"Notebook {name} Part {i}" for i in range(1, num_tasks+1)])
    task_args += (nb_args := [{"name": name, "i": i, "outline": outline} for i in range(1, num_tasks+1)])
    ## Notebook-level parallelization: Bottlenecks down to one notebook at a time
    # task_vals += RunnableLambda(summarize_section).batch(nb_args)

## All-at-once parallelization: Bottleneck is the set maximum concurrency (since threads are hardware-limited)
task_vals = RunnableLambda(summarize_section).batch(task_args)

<details>
<summary><b>并行化注意事项：</b></summary>

- 如果您想在工作流的输入之间实现并行化，`batch` 是一个很好的选择，它已经包含了线程安全和并发限制（您可以通过观察第一个线程完成的时间来估计最大并发数）。如果您想在不同的链之间并行化（也就是说，您希望相同的数据同时通过多个管道），可以使用 `RunnableParallel`。这两者都比带信号量或线程池的方法更简单（但也更不灵活），后者允许更自定义化的处理方式。

- 我们调整了提示词和数字，试图为每个部分生成摘要，但请注意，当 notebook 部分不足时，“总结第 i 部分”的目标是定义不清的。虽然这对我们在评估的目的来说已经足够，但在整体上并不是理想的。稍后您会学习一些策略来缓解这种“语言输入 -> 语言输出”的问题（***提示***：参见**结构化输出**）。

</details>

**保存结果**

为了帮助可视化这些结果，您可以运行以下代码单元将其保存到 JSON 文件中。这些导出用于构建示例聊天机器人，因此看看这个工作流的输出会很有趣。

In [ ]:
## Note, we're going to need these generations for the assessment, so save them here
nbsummary = {
    "course": "NVIDIA Deep Learning Institute's Instructor-Led Course called \"Rapid Application Development with Large Language Models\"",
    "summary": short_summary, 
    "filenames": filenames
}

for i, (name, outline) in enumerate(zip(filenames, running_summaries)):
    if not name.endswith(".ipynb"): continue
    nbsummary[name] = {"outline": outline}
    task_iter = range(i*num_tasks, (i+1)*num_tasks)
    nbsummary[name]["sections"] = [task_vals[j] for j in task_iter]
        
import json
json.dump(nbsummary, open('notebook_chunks.json', "w"), sort_keys=True, indent=4)

# with open('notebook_chunks.json', 'r') as fp:
#     nbsummary = json.load(fp)

#### **超越模型先验的进展**

这些公式使大语言模型不仅能推理，还能生成任意长或任意短的响应。当以编程方式实现时，它们还使您可以控制上下文范围和瓶颈，在轻量但范围受限和信息充分但缓慢过程之间进行权衡。

然而，这些技术本身只是建立在语言模型之上的工程，局限于其即时上下文、提示词和先验知识。**LLM 最大的潜力不在于模型权重和预计算工作流中的静态内容，而在于与丰富的环境结合，引导其推进并按照响应行动。**

<hr>
<br>

## **7.2：工具简介**

**工具**通过将外部数据源、计算工具和动态系统整合到生成过程中，将这项技术提升到一个新水平。关键是，利用其内部参数以外的额外资源，可以进一步适应、检索、学习，从而在编排层面上影响环境。

### **工具的基本原理：**

为了使 LLM 能与外部环境交互，必须发生以下**一种**情况：
- **观察系统：**其上下文可以通过一些程序化查询进行丰富，这些查询会根据应用状态而变化。
    - **[可选]**在接收到依赖状态的上下文后，它能够改变应用状态。
- **交互系统：**它必须能够向一个环境发送查询，该环境会响应其请求并改变应用状态。
    - **[可选]**在请求后，它会从环境中收到反馈。

**观察系统很容易理解，**因为唯一的要求是一些语义上可理解的上下文。用一些变量填充提示词，比如目录中的文件名，这样就有了一个能告诉您文件信息的 LLM 组件。允许用户选择要输入 LLM 的文档，这时它就是在与用户的选择进行推理。这里的难点不在于“如何以编程方式构建上下文”，而在于“放入什么”，难点是在选择上。

**相比之下，交互系统的属性难以强制执行，**因为我们的 LLM 输出是非结构化的。上一部分中，我们的 LLM 之所以能与其它 LLM 交互，是因为它们都是文本输入和输出，但映射到普通非 LLM 工具就可能会面临挑战了，对吧？


#### **语法强制（Grammar Enforcement）**

幸运的是，LLM 采样工具和编排工具在努力弥合这一鸿沟，允许我们通过一种叫做**语法强制**的方式直接调用工具！

回想一下我们在早期部分讨论的工作流，算法输入模式自然定义为键值对字典：

```python
{
    "arg1": value1,
    "arg2": value2,
    ...
}
```

从 LLM 的角度来看，假设变量名和值是人类可理解的，就可以毫无障碍地将其作为上下文输入。然而，生成这样的模式就没那么简单了。开发者们早早意识到，虽然您可以“请求”一个 LLM 生成“有效的 python 代码”或“仅包含正确键的有效 JSON”，但这些策略通常需要后处理，并且失败的情况会时常出现。

如今，许多模型支持**“模式”**输入，指定输出所需的格式。考虑以下工具实例化，它创建了一个自然适配该接口的对象：

In [ ]:
from langchain.tools import tool

@tool
def add(
    explanation_of_what_the_user_wants: str, ## Optional. Gives some food for thought.
    a: float, 
    b: float
) -> int:
    """Adds a and b. Requires both arguments. Can be repurposed for subtraction"""
    return a + b

print(add.name)
print(add.description)
print(add.args)

当输入模式给到 LLM 服务时，生成必须在采样层面上遵循所需的语法。换句话说，无论从每次自回归调用生成的整体概率向量如何：
- 前几个 token 必须是 `{'a': `，按此顺序。
- token 范围将限制在有效 token 的某个子集 `0123456789.e-+}` 之内，直到生成 `}`。
- 接下来的 token 采样必须是 `}, {'b': `。
- 一直重复，最后的 `}` 是强制停止 token。

鉴于我们的 `tool` 组件在构建过程中会自动聚合输入模式等细节，我们可以简单地使用 `with_structured_output` 将输入模式绑定到连接器，然后假设其调用必须遵循。

下面是一个示例，附带一些重要的注意事项：

- LLM 本身并不知道这种语法强制，若不加以控制可能会出错。因此如果没有更一般的格式遵循指令，比如默认的 LangChain 指令格式字符串，那么给 LLM 提供参数模式是个好习惯。

- 除了语法强制，Llama 和其它类似模型[**明确以支持函数调用为目标进行训练**](https://github.com/meta-llama/llama-models/blob/6ad6fd6bb8f5fc841acecc2e48958eee25ff3b1c/models/llama3_1/prompt_format.md?plain=1#L306)。您会在后面的部分注意到，语法强制可能和 `<function=foo>{}</function>` 这样的输出配合使用一个解析器。

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from random import random

add_tool = (
    llm.with_structured_output(add.input_schema).bind(temperature=0)
    | dict
    | {
        "args": RunnablePassthrough(),
        "result": add,
    }
)

for i in range(10):

    a, b = random() * 10e10, random() * 10e10 
    # a, b = round(a), round(b)
    
    tool_output = add_tool.invoke([
        ("system", PydanticOutputParser(pydantic_object=add.input_schema).get_format_instructions()),
        # ("system", f"Assume schema of {add.args}"),  ## Lighter reinforcement.
        ("user", f"Add the values of {a} and {b}."),
    ])

    tool_a = tool_output.get("args").get("a")
    tool_b = tool_output.get("args").get("b")
    tool_result = tool_output.get("result")
    
    print("[PASSED]" if (a+b) == tool_result else "[FAILED]", end=" ")
    print(f"{tool_a} + {tool_b} = {tool_result} vs {a + b}" )
    
    if "explanation_of_what_the_user_wants" in tool_output.get("args"):
        print("\tThoughts:", tool_output.get("args").get("explanation_of_what_the_user_wants"))

**试试:**
- 如果您不提供系统消息，会发生什么？它还会有效吗？
- 如果您将系统消息移回用户消息中，会发生什么？它也能正常工作吗？
- 如果您不包含 `explanation_of_what_the_user_wants` 变量，会发生什么？效果是更好还是更差？

从微观层面上看，您让 LLM 做了加法！这其实……说实话也没什么大不了的，对吧？好吧，就算在这个狭义的范围内，这可能也比让 LLM 为您做这件事要好……

In [ ]:
for i in range(10):

    a, b = random() * 10e10, random() * 10e10 
    a, b = round(a), round(b)
    
    tool_output = (llm | StrOutputParser()).invoke([
        ("user", f"Add the values of {a} and {b}. Only return the final answer, not the arithmetic"),
    ])
    
    print("[PASSED]" if str(a+b) in tool_output else "[FAILED]", end=" ")
    print(f"{a} + {b} = {tool_output} vs {a + b}" )

<br>

但从宏观层面来看，我们可以扩展结构化输出生成，让我们的 LLM 遵循几乎任何预定义的输出模式。这项能力对于将 LLM 视为更大软件工作流的核心组件至关重要。

<hr>
<br>

## **7.3：** 多工具智能体系统

我们之前提到，和环境交互需要一个系统，这个系统要么能观察并推理动态环境，要么能直接影响并对其做出反应。能够同时做到这两点的系统被称为**智能体**。

> 更一般地说，如果它能够观察、思考、反应，并根据个人指令对环境采取行动，。
>
> 换句话说，如果它能根据当前状态选择工具或行动，利用它影响某些事物，并理解其决策如何推动其朝着目标前进，这个系统就是***智能体***。

**至少，智能体通常只需要一个组件：** 
- 一个选择路径的路由机制。

**除此之外，许多智能体还包括：**
- 对所选路径参数的预测（如果需要）。
- 一个用来积累记忆的缓冲区（如果需要）。
- 一个总体或选择性应用的指令（如果需要）。

<div><img src="imgs/simple-agent.png" width="600"/></div>

<br>

按照这个定义，下面的示例在技术上是一个基本的智能体循环，至少满足作为智能体的要求：

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch
from random import random

sys_msg = (
    "Please help the user. After every response, output '[stop]` if the conversation should end and [pass] otherwise."
)

prompt = ChatPromptTemplate.from_messages([("system", sys_msg), ("placeholder", "{messages}")])
chain = prompt | llm | StrOutputParser()

state = {"messages": []}
agent_msg = ""

while True:
    try: 
        if "[stop]" in agent_msg: break
        else: pass
        
        ## TODO: Update the messages appropriately
        human_msg = input("\n[Human]:")
        state["messages"] += [("human", human_msg)]

        ## Initiate an agent buffer to accumulate agent response
        agent_msg = ""
        print("\n[Agent]: ", end="")
        ## TODO: Stream the LLM's response directly to output and accumulate it
        for token in chain.stream(state):
            agent_msg += token
            print(token, end="")

        ## TODO: Update the messages list appropriately
        state["messages"] += [("ai", agent_msg)]
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        break

<br>

在我们的智能体抽象中，上面的循环使用了一个简单的*“输出是否包含 `[stop]`”*的启发式来决定是否继续对话。这算不算微不足道？是的，从技术上讲，这只是一个路由机制 `"[stop]" in agent_msg` 和随后的工具调用（`break`），但其逻辑延伸却出乎意料地强大！

- **[多工具]**如果我们有比 `pass` 和 `break` 更多的工具会怎样？
- **[状态管理]**如果我们的工具选择（以及工具的参数选择）改变了系统的行为呢？
- **[检索]**如果我们的工具为上下文提供了相关信息呢？
- **[智能多模态]**如果我们的工具允许智能体在必要时输出其它模态（图像、音频、视频等）呢？

### **一个简单的多工具智能体**

在上面提到的各个类中，**多工具智能体**是最包罗万象的，因为几乎所有的例子*都*可以归类于此。为了探索这个概念，让我们创建一个具有以下工具的计算器智能体：

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> int:
    """Adds a and b. Requires both arguments."""
    return a + b

@tool
def subtract(a: float, b: float) -> int:
    """Subtracts a and b. Requires both arguments."""
    return a + b

@tool
def multiply(a: float, b: float) -> int:
    """Multiplies a and b. Requires both arguments."""
    return a * b

@tool
def divide(a: float, b: float) -> int:
    """Divides a by b. Requires both arguments."""
    return a / b

@tool
def power(a: float, b: float) -> int:
    """Raises a to the power of b. Requires both arguments."""
    return a ** b

@tool
def no_tool() -> str:
    """Null tool; says no tool should be used"""
    return "No Tool Selected"

如果我们想的话，可以创建一个路由机制，首先预测使用哪个工具，然后调用该工具。然而，这个过程在许多系统中是如此常见且有用，以至于很多系统在服务端以类似于结构化输出的方式支持它：


In [ ]:
from langchain_core.runnables import Runnable, RunnableAssign, RunnablePassthrough, RunnableLambda
from langgraph.prebuilt import ToolNode


math_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a math bot! Help the user as much as possible."),
    ("placeholder", "{messages}"),
])

toolset = [
    add,
    multiply,
    divide,
    power,
    no_tool,
]

## Create a client-side resolved which executes the tool picked by the server.
tool_node = ToolNode(toolset)

simple_chain = (
    math_prompt 
    ## Bind the tools to the connector, effectively feeding in the possible schemas on every invocation.
    | llm.bind_tools(toolset)
    # | {"messages": lambda x: [x]} | tool_node | RunnableLambda(lambda x: x.get("messages"))
)
simple_chain.invoke({"messages": [("user", "What's 56766*30432?")]})

In [ ]:
question = "What's 333333*555555?"
print(f"\nSimple {question = }")
print(simple_chain.invoke({"messages": [("user", question)]}))

question = "What's 333333*555555 and 444444+222222?"
print(f"\nDouble {question = }")
print(simple_chain.invoke({"messages": [("user", question)]}))

question = "What's 555555 times (444444 plus 222222)?"
print(f"\nComplex {question = }")
print(simple_chain.invoke({"messages": [("user", question)]}))

question = "Introduce yourself in 10 words or less!"
print(f"\nRandom {question = }")
print(simple_chain.invoke({"messages": [("user", question)]}))

In [ ]:
# llm._client.last_inputs

**从这个演示中可以看到：**
- 这个服务部署中的内部路由器一次只能够思考一个工具调用。
- 这个内部路由实际上并没有向模型通报其模式。它只是在生成时强制执行这些模式。
- 您可能偶尔会看到生成内容而不是 `no_tool` 调用。这意味着除了客户端的 no_tool 支持外，还有一个服务端版本，它改变了端点对于非结构化生成的行为。

### **[进阶]** 会话工具调用

通过一些更先进的 LLM 编排范式，我们可以在默认的工具调用实现上进行改进，创建一个能够在看似单一的生成中同时进行**会话和工具调用**服务。这是一个较为复杂的话题，我们不会详细讨论，但提到它有几个重要原因：
- 与服务端智能能力相关的直觉支撑了许多更高级的自定义功能，如工具感知的端点、服务端反思/思维链，以及特定入口的知识库。
- 在评估中，我们希望利用一个更简化的智能框架，称为 LangGraph，它使得会话工具调用变得更简单。通过构建和激励这个模块，我们将能够与像 OpenAI 的 GPT4 这样的模型保持代码的一致性。

以下代码块介绍了一个自定义的 `ConversationalToolCaller` 组件，旨在说明如何将一个定制的工具调用方式转变为一个简化的入口：

In [ ]:
from chatbot.conv_tool_caller import ConversationalToolCaller

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a math bot! Help the user as much as possible by using the tools to answer their questions."
        " Think step-by-step, and work out your math using the tools provided."
    )),
    ("placeholder", "{messages}"),
])

tool_instruction = (
    "You have access to the tools listed in the toolbank. Use tools only within the \n<function></function> tags."
    " Select tools to handle uncertain, imprecise, or complex computations that an LLM would find it hard to answer."
    " You can only call one tool at a time, and the tool cannot accept complex multi-step inputs."
    "\n\n<toolbank>{toolbank}</toolbank>\n"
    "Examples (WITH HYPOTHETICAL TOOLS):"
    "\nSure, let me call the tool in question.\n<function=\"foo\">[\"input\": \"hello world\"]</function>"
    "\nSure, first, I need to calculate the expression of 5 + 10\n<function=\"calculator\">[\"expression\": \"5 + 10\"]</function>"
    "\nSure! Let me look up the weather in Tokyo\n<function=\"weather\">[\"location\"=\"Tokyo\"])</function>"
)

tool_prompt = (
    "You are an expert at selecting tools to answer questions. Consider the context of the problem,"
    " what has already been solved, and what the immediate next step to solve the problem should be."
    " Do not predict any arguments which are not present in the context; if there's any ambiguity, use no_tool."
    "\n\n<toolbank>{toolbank}</toolbank>\n"
    "\n\nSchema Instructions: The output should be formatted as a JSON instance that conforms to the JSON schema."
    "\n\nExamples (WITH HYPOTHETICAL TOOLS):"
    "\n<function=\"search\">[\"query\": \"current events in Japan\"]</function>"
    "\n<function=\"translation\">[\"text\": \"Hello, how are you?\", \"language\": \"French\"]</function>"
    "\n<function=\"calculator\">[\"expression\": \"5 + 10\"]</function>"
)

conv_llm = ConversationalToolCaller(
    tool_instruction=tool_instruction, 
    tool_prompt=tool_prompt, 
    llm=llm
).get_tooled_chain()

agent_chain = agent_prompt | conv_llm.bind_tools(toolset)

response1 = agent_chain.invoke({"messages": [("user", "What's (56766*30432+3043)/99?")]})
print(repr(response1))

如果是流式传输，上面的代码将首先与消息相关联生成一个 token，然后在保持语法约束的同时转储工具调用的函数参数。当不是流式传输时，响应将按预期一次性返回。

在这个调用之后，常见的做法是使用 ToolNode 组件来去除消息中的工具调用，并将每个工具调用与合适的执行器配对，以产生正确的输出。可以按如下方式完成：

In [ ]:
from langgraph.prebuilt import ToolNode

tool_caller = RunnableLambda(lambda string: ToolNode(toolset).invoke({"messages": [string]})["messages"])

tool_caller.invoke(response1)

这段代码可在 [`conv_tool_caller.py`](conv_tool_caller.py) 中找到，详细的拆解超出了课程的范围。然而，我们仍推荐您花时间看看，因为其中有关于通过入口传输流和接受客户端访问的自定义参数的不错的信息。

<hr>
<br>

## **[练习] 7.4：启用智能循环**

现在我们有了一个会话工具调用入口，接下来就可以创建一个类似**智能循环**的东西了。
- 在之前的 notebook 中，我们设定了一个**简单的聊天循环**，可以让我们积累聊天记录。
- 在这个 notebook 中，我们将设置一个**多步骤智能循环**，允许我们的系统在给用户返回最终答案前调用多个工具。

<div><img src="imgs/basic-react.png" width="600"/></div>

我们将选择实现 [**ReAct (Reason+Act)** 循环](https://arxiv.org/abs/2210.03629)，它做了一个简单的假设：

**不断调用工具并观察工具调用的结果，直到得到最终答案。**

有时这可以很明确，比如能实际给出“最终答案”和“询问用户”的工具；而其它时候，没调用工具时跳过用户也隐含地表明了这一点。我们将选择后者，因为入口现在支持会话和工具调用，所有这些都在一个请求中完成。 

In [ ]:
from langchain_core.messages import ToolMessage

state = {"messages": []}
agent_results = []

## BEGIN EXERCISE

while True:
    try: 
        ## TODO: If a tool is not called, the answer-generating loop ends. 
        ##   When this happens, ASK THE USER FOR A NEW INPUT.
        if not agent_results:
            state["messages"] += [("human", input("\n[Human]:"))]

        print("\n[Agent]: ", end="")
        agent_response = None
        for chunk in agent_chain.stream(state):
            agent_response = chunk if not agent_response else agent_response + chunk
            print(chunk.content, end="")
        print()

        ## Get the agent message (pre-tool call), the function arguments, and the call invocation 
        agent_fncalls = [call.get("function") for call in agent_response.additional_kwargs.get("tool_calls", [])]
        agent_results = [result.content for result in tool_caller.invoke(agent_response)]
        if agent_fncalls: print(agent_fncalls)
        if agent_results: print(agent_results)

        ## TODO: If a tool is called, record it in the conversational history.
        if not agent_results:
            response = agent_response.content
        else: 
            response = (
                f"{agent_response.content}\n"
                f"\n<RESULT>{agent_results}</RESULT>"
            )
        
        state["messages"] += [("ai", response)]
        
    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        break

**潜在问题：** 
- (56766*30432+3043)/99 是多少？

**寻找答案的过程：**
- 第一步：1727502912
- 第二步：1727505955
- 第三步：17449555.101010103

**挑战问题：**
- 计算前 25 个斐波那契数。
- 现在使用工具计算第 26 到 30 个数字。
- 第 30 个数字比第 25 个数字大多少？

<hr>
<br>

# <font color="#76b900">**总结**</font>

这部分，我们介绍了有状态的 LLM 系统以及支撑更复杂和可控（甚至试图自我控制）模型的逻辑！这仅仅是您可以使用 LLM 做的令人兴奋事情的开始，希望您能喜欢！

从有限的任务特定编码器到强大的生成模型和自我引导模型的转变，希望您能意识到我们讨论的每个模型在整个架构中都有其位置！无论是关键的基础组件，具性价比的补充机制，还是进一步发展的灵感来源，或者是该领域进步的基石，尽量在今后的工作中保留这些工具，并使用那些最适合您环境的工具！

**在下一个，也是最后的 notebook 中，我们将请您创建一个更专业的工作流，以便练习并扩展之前的技术！**

In [ ]:
# ## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)